Importing libraries

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import re
from json.decoder import JSONDecodeError
import datetime

Basic Function Definition

Basically are function that validate if value is null or empty, converts to numeric some string values

On another hand retrieving information about dates, like months, days and year

In [ ]:
import datetime
def isNaN(num):
    if num == '':
      return False
    return num != num
def convert_numeric(value):
   #print("value ", value," and type is: ", type(value))
   if value is None:
      return 0
   if isinstance(value, (int, float, complex)) and not isinstance(value, bool) == True:
        return value
   if(type(value) == str):
      try:
        #print("return values is: ",value)
        return float(value)
      except:
        #print("Catching value: ",value)
        return 0
   if value == np.nan or value  == "":
        return 0
   else:
      return 0
def get_month_numeric(str_month):
    month_converted = str_month.lower()
    match month_converted:
        case 'enero':
            return 1
        case 'febrero':
            return 2
        case 'marzo':
            return 3
        case 'abril':
            return 4
        case 'mayo':
            return 5
        case 'junio':
            return 6
        case 'julio':
            return 7
        case 'agosto':
            return 8
        case 'septiembre':
            return 9
        case 'octubre':
            return 10
        case 'noviembre':
            return 11
        case 'diciembre':
            return 12
        case _:
            return 0   # 0 is the default case if month is not found
        
def get_spanish_day(str_day):
    str_day = str_day.lower()
    match str_day:
        case 'monday':
            return 'lunes'
        case 'tuesday':
            return 'martes'
        case 'wednesday':
            return 'miercoles'
        case 'thursday':
            return 'jueves'
        case 'friday':
            return 'viernes'
        case 'saturday':
            return 'sabado'
        case 'sunday':
            return 'domingo'        
        case _:
            return ""   # empty is the default case if day is not found
        
def get_month(date):
  try:
      
      movie_month = datetime.datetime.strptime(date, "%d/%m/%Y").month
      
      #return movie_month
  except:
      print("Exception:", date)
      movie_month = 0
  return movie_month
  
def get_day(date):
  try:
      date_movies = datetime.datetime.strptime(date, "%d/%m/%Y")      
      day_en= date_movies.strftime("%A").lower()      
      day_es = get_spanish_day(day_en)      
  except:
      print("Error in: ",day_en)
      day_es = ""
  return day_es

def get_year(date):
  try:
      date_movies = datetime.datetime.strptime(date, "%Y-%m-%d")
      year_str = date_movies.year
  except:
      year_str = ""
  return year_str
  


JSON functions

Below function are interacting with json elements on spread sheet, then some function validate and format string to json though patterns and regex expressions

In [14]:
def get_year(date):
  try:
      date_movies = datetime.datetime.strptime(date, "%Y-%m-%d")
      year_str = date_movies.year
  except:
      year_str = ""
  return year_str

def jsonLoadsProduComp(production_companies):
  pattern3 = r'(\w*\s|\"|\"\w*\s)\"(\w*|\w*\s\w*)\"(\s|\")'
  pattern2 =  r'(\"\w+\s\w|\"\w+|\s\w|\w|\"\w+\s|\w+\s)\"(\d|\w+\s\w+\"|\w*\"|\s|\w\s|\w+\d|\w+)'
  patternInitSimpleQuote = r'(\s|\s\w*)\'(\w*\s\w*)\'(\s|\w*\s)'
  patternInitDoubleQuote = r'(\w*\s)\"(\w*)\"(\w*)'
  dicc_production_companies = {}
  for indx,val in production_companies.items():
      if isNaN(val) ==  False and val != "[]" and val != "False":
        x = re.sub(patternInitSimpleQuote,r'\1¬\2¬\3',val)
        x = re.sub(patternInitDoubleQuote,r'\1¬\2¬\3',x)
        x = x.replace(u'\\xa0', u' ')
        x = x.replace("\"\"", "¬")
        x = x.replace("\'\'", "¬")
        x = x.replace("'", "\"")

        x = re.sub(pattern3,r'\1¬\2¬\3',x)

        x = x.replace("None", "\"\"")
        x = x.replace("¬", "\'")

        try:
          val = json.loads(x, strict=False)
        except JSONDecodeError as e:
          x = val.replace("'", "\"")
          x = x.replace("\"\"", "\"")
          x = re.sub(pattern2,r'\1¬\2',x)
          x = x.replace("¬", "\'")
          val = json.loads(x, strict=False)

        except (ValueError, TypeError):
          print(x)
          print("An ValueError exception occurred: ",ValueError.args, "TypeError:",TypeError.with_traceback)
        except:
          print(x)
          print("An exception occurred")

      dicc_production_companies[indx] = val
  return pd.Series(dicc_production_companies)

def jsonLoadsCast(cast):
  dicc_belongs_collection = {}
  for indx,val in cast.items():
      if val != '':
        x = val.replace("'", "\"")
        x = x.replace("None", "\"\"")
        try:
          json_val = json.loads(x)
        except:
          #print(x)
          try:
            x = re.sub(r'\"(\w*\s)\"(\w*)\"(\s\w*)\"',r'"\1¬\2¬\3"',x)
            json_val = json.loads(x)
          except:
            try:
              #print(x)
              x = re.sub(r'\"(\w*)\"(\s*\w*\s\w*\s*\w*)\"',r'"\1¬\2"',x)
              #print(x)
            except:
              #print(x)
              x = re.sub(r'\"(\w*)\"(\s*\w*\s\w*\s*\w*)\"',r'"\1¬\2"',x)
              #print(x)
              json_val = json.loads(x)

      dicc_belongs_collection[indx] = json_val
  return pd.Series(dicc_belongs_collection)

def json_full_search(lookup_key, json_dict, search_name_result = []):
    if type(json_dict) == dict:
        for key, value in  json_dict.items():
            if key == lookup_key:
                search_name_result.append(value)
            json_full_search(lookup_key, value, search_name_result)
    elif type(json_dict) == list:
        for element in json_dict:
            json_full_search(lookup_key, element, search_name_result)
    return search_name_result

def jsonLoadsCrew(crew):
  dicc_belongs_collection = {}
  json_val = ""
  for indx,val in crew.items():
      if val != '':
        x = val.replace("'", "\"")
        x = x.replace("None", "\"\"")
        try:
          json_val = json.loads(x)
        except:
          #print(x)
          try:
            x = re.sub(r'\"(\w*\s)\"(\w*)\"(\s\w*)\"',r'"\1¬\2¬\3"',x)
            json_val = json.loads(x)
          except:
            try:
              #print(x)
              x = re.sub(r'\"(\w*)\"(\s*\w*\s\w*\s*\w*)\"',r'"\1¬\2"',x)
              #print(x)
            except:
              #print(x)
              x = re.sub(r'\"(\w*)\"(\s*\w*\s\w*\s*\w*)\"',r'"\1¬\2"',x)
              #print(x)
              json_val = json.loads(x)

      dicc_belongs_collection[indx] = json_val
  return pd.Series(dicc_belongs_collection)




In [19]:
def json_job_search(search_key, search_position, actor, json_dict, search_name_result = []):
    if type(json_dict) == dict:
        for key, value in  json_dict.items():
            if key == search_key:
                if search_position == value:
                  search_name_result.append(json_dict['name'])
            json_job_search(search_key, search_position, actor, value, search_name_result)
    elif type(json_dict) == list:
        for element in json_dict:
            json_job_search(search_key,search_position,  actor, element, search_name_result)
    return search_name_result

In [15]:
def jsonLoadsBelong(belongs):
  dicc_belongs_collection = {}
  for indx,val in belongs.items():
      if val != '':
        x = re.sub(r'(\w|\w\s)[\'](\s|\w)',r'\1¬\2',val)
        x = x.replace("'", "\"")
        x = x.replace("None", "\"\"")
        x = x.replace("¬", "\'")
        val = json.loads(x)
      dicc_belongs_collection[indx] = val
  return pd.Series(dicc_belongs_collection)

Aditional Functions

These functions are involve on customer specification

In [ ]:
def get_return(revenue, budget):
  new_revenue = convert_numeric(revenue)
  new_budget = convert_numeric(budget)
  if (new_revenue == 0 or new_budget == 0):
    return 0
  val = new_revenue/new_budget
  #print (val)
  return val



Formatting Dataset Movies, fill na, updating formats and dropping columns not necesaries

In [ ]:
def format_dataset_movies(movies_dataset):
  print("Starting ...")
  format = '%Y-%m-%d'
  y = ['0','1','12']

  movies_dataset['belongs_to_collection'] = movies_dataset['belongs_to_collection'].fillna("")
  movies_dataset['genres'] = movies_dataset['genres'].apply(lambda x: json.loads(x.replace("'", "\"")))

  movies_dataset['belongs_to_collection_json'] = jsonLoadsBelong(movies_dataset['belongs_to_collection'])
  movies_dataset['production_companies_json'] = jsonLoadsProduComp(movies_dataset['production_companies'])

  movies_dataset['revenue'] = movies_dataset['revenue'].fillna("0")
  movies_dataset['budget'] = movies_dataset['budget'].fillna("0")

  movies_dataset['release_date_clean'] = movies_dataset['release_date'].fillna("0")
  movies_dataset['release_date_clean_format']=movies_dataset['release_date_clean'].apply(lambda x: datetime.datetime.strptime(x, format) if (x not in(x)) else x)


  movies_dataset['retorno']=movies_dataset.apply(lambda row: get_return(row['revenue'], row['budget']), axis=1)
  movies_dataset = movies_dataset.drop(columns=['video','imdb_id','adult','original_title','poster_path', 'homepage'])

  movies_dataset['month_movies']=movies_dataset.apply(lambda x: get_month(x.release_date_clean_format), axis=1)
  movies_dataset['day_movies']=movies_dataset.apply(lambda x: get_day(x.release_date_clean_format), axis=1)
  movies_dataset['year_movies']=movies_dataset.apply(lambda x: get_year(x.release_date_clean_format), axis=1)
  movies_dataset['key_title']=movies_dataset.apply(lambda x: x.title.lower() if isinstance(x.title, str) else x.title , axis=1)

  return movies_dataset



Formatting DataSet Credits

In [ ]:
def format_dataset_credits(credits_dataset):    
    credits_dataset['cast_json'] = jsonLoadsCast(credits_dataset['cast'])
    credits_dataset['crew_json'] = jsonLoadsCrew(credits_dataset['crew'])
    credits_dataset = credits_dataset.drop(columns=['crew','cast'])
    return credits_dataset